Step: Load data and packages

In [1]:
%load_ext lab_black

In [3]:
%cd ..

/mnt/batch/tasks/shared/LS_root/mounts/clusters/mhanauer60/code/Users/mhanauer/ml-text-cleaning-py


In [4]:
import pandas as pd
import pyodbc
import numpy as np
from skimpy import clean_columns
from pyprojroot import here
import os
import docx
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np

nltk.download("punkt")
nltk.download("stopwords")
path_data = here("./data")
os.chdir(path_data)

from azureml.core import Workspace, Dataset

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
workspace = Workspace.from_config()

datastore = workspace.get_default_datastore()

ModuleNotFoundError: No module named 'docx'

Step: Combine text by case id, and lower text

In [5]:
import PyPDF2
import docx
import numpy as np
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer


from PyPDF2 import PdfReader


def read_pdf_file(file_path):
    with open(file_path, "rb") as file:
        reader = PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text


def read_word_file(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text
    return text

In [6]:
cv_path = "./CV_Matt_Hanauer.docx"
resume_pdf_path = "./resume_matt_hanauer.pdf"
resume_path = "./resume_matt_hanauer.docx"
job_description_path = "./job_example.docx"

resume_text = read_word_file(resume_path)
resume_pdf = read_pdf_file(resume_pdf_path)
cv_text = read_word_file(cv_path)
job_description_text = read_word_file(job_description_path)

In [41]:
def preprocess_text(text):
    words = word_tokenize(text.lower())
    words_filtered = [word for word in words if word.isalnum()]
    return words_filtered


def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)


def compare_texts(text1_tokens, text2_tokens, top_n_scores=10):
    text1_sentences = " ".join(text1_tokens)
    text2_sentences = " ".join(text2_tokens)

    text1_embedding = model.encode(text1_sentences, convert_to_tensor=True)
    text2_embedding = model.encode(text2_sentences, convert_to_tensor=True)

    similarity_score = cosine_similarity(text1_embedding, text2_embedding).round(2)
    return similarity_score

In [8]:
compare_texts(resume_text, job_description_text)

0.81

In [9]:
compare_texts(resume_pdf, job_description_text)

0.84

In [10]:
compare_texts(cv_text, job_description_text)

0.89

Step: Identify which key words are missing

In [42]:
from nltk.corpus import stopwords
import nltk
from nltk.probability import FreqDist

# Load the English stop words
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


def calculate_word_frequency(text_tokens):
    filtered_tokens = [token for token in text_tokens if token not in stop_words]
    fdist = FreqDist(filtered_tokens)
    return fdist

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
# Preprocess the texts
resume_tokens = preprocess_text(resume_text)
job_description_tokens = preprocess_text(job_description_text)

In [44]:
# Calculate frequency distributions
resume_fdist = calculate_word_frequency(resume_tokens)
job_description_fdist = calculate_word_frequency(job_description_tokens)

In [45]:
# Get the top 10 words from the job description
top_n = 10
top_job_description_words = [
    word for word, freq in job_description_fdist.most_common(top_n)
]

# Find the missing top 10 words in the resume
missing_keywords = []
for word in top_job_description_words:
    if word not in resume_fdist:
        missing_keywords.append(word)

missing_keywords

['affairs', 'gsk', 'patients', 'innovative']